In [1]:
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import re
from selenium.common.exceptions import TimeoutException


In [2]:
URL = 'https://hh.ru'
profession = 'Программист'

service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(URL)
driver.implicitly_wait(1)
driver.set_page_load_timeout(10)

headers = {
    'Accept': '*/*',
    'User-Agent': 'Custom'
}



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\dm1tr\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [3]:
src = driver.page_source
soup = BeautifulSoup(src, 'lxml')

In [4]:
main_block = soup.find_all('div', class_='dashboard-tiles-item__body')

for b in main_block:
    print(b)
    print()

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Вакансии дня</div><div class="dashboard-tiles-item__salary"></div></div><div class="dashboard-tiles-item__counter"> </div></div>

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Компании дня</div><div class="dashboard-tiles-item__salary"></div></div><div class="dashboard-tiles-item__counter"><span>1932<!-- --> <!-- -->вакансии</span></div></div>

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Работа из дома</div><div class="dashboard-tiles-item__salary"></div></div><div class="dashboard-tiles-item__counter"><span>37100<!-- --> <!-- -->вакансий</span></div></div>

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Подработка</div><div class="d

In [5]:
for sub in main_block:
    if sub.find('div', class_='dashboard-tiles-item__title').text == profession:
        profession_selected_block = sub
print(profession_selected_block)

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Программист</div><div class="dashboard-tiles-item__salary">до <!-- -->505 000<!-- --> <!-- -->руб.</div></div><div class="dashboard-tiles-item__counter"><span>18280<!-- --> <!-- -->вакансий</span></div></div>


In [6]:
from selenium.webdriver.support.wait import WebDriverWait

professions = driver.find_elements(by=By.CLASS_NAME, value='dashboard-tiles-item__title')

links_to_profession = list()

if professions:
    professions[0].click()
    professions[0].click()

for elem in professions:
    if elem.text == profession:
        elem.click()
        element = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, "//*[@class='multiple-column-list multiple-column-list_wide']"))
        )
        soup = BeautifulSoup(driver.page_source, 'lxml')
        ul = soup.find(class_='multiple-column-list multiple-column-list_wide')
        for li in ul:
            links_to_profession.append(li.a.get('href'))

In [7]:
def scrap_offer(link, data):
    driver.get(link)
    driver.implicitly_wait(1)
    src = driver.page_source

    soup = BeautifulSoup(src, parser='lxml')
    header = soup.find('div', class_='vacancy-company-wrapper')
    reviews_button = header.find('button', class_='_2kMmwZ7___bloko-link _2iCEfb3___bloko-link_pseudo')
    if reviews_button:
        reviews_number = reviews_button.span.text.split()[0]
        rating = header.find('div', class_='_2uXXPFp___bloko-text _3Iu9HPD___bloko-text_extra-large mmwQivW___bloko-text_strong').text
    else:
        reviews_number = None
        rating = None

    data['reviews_count'].append(reviews_number)
    data['avg_rating'].append(rating)


In [ ]:
data = {'prof_type': list(), 'salary': list(), 'reviews_count': list(), 'avg_rating': list()}
page = 0
for link in links_to_profession[:3]:
    req = requests.get(f'{URL}{link}&page={page}', headers={'User-Agent': 'Custom'})

    while req.status_code == 200:
        soup = BeautifulSoup(req.text, parser='lxml')

        prof_type = soup.find('h1', class_='bloko-header-section-3').text
        prof_type = re.compile('«.*»').findall(prof_type)[0][1:-1]

        for offer in soup.find_all('div', class_='vacancy-serp-item'):
            raw_salary = offer.find('span', class_='bloko-header-section-3')

            data['prof_type'].append(prof_type)

            if not raw_salary:
                data['salary'].append(None)
            else:
                salary = re.compile('\d+').findall(raw_salary.text.replace(' ', ''))

                if len(salary) > 1:
                    data['salary'].append(f'{salary[0]}-{salary[1]}')
                elif 'от' in raw_salary.text:
                    data['salary'].append(f'от {salary[0]}')
                elif 'до' in raw_salary.text:
                    data['salary'].append(f'до {salary[0]}')
                else:
                    data['salary'].append(None)

            offer_link = offer.find('a', class_='bloko-link').get('href')
            try:
                scrap_offer(offer_link, data)
            except TimeoutException:
                print('Timeout')


        page += 1
        req = requests.get(f'{URL}{link}&page={page}', headers={'User-Agent': 'Custom'})
    page = 0



df = pd.DataFrame(data=data)

In [9]:
df

,prof_type,salary
0,Программист 1С,от 200000
1,Программист 1С,200000-250000
2,Программист 1С,140000-200000
3,Программист 1С,250000-300000
4,Программист 1С,от 250000
...,...,...
1946,Программист C#,от 50000
1947,Программист C#,None
1948,Программист C#,None
1949,Программист C#,None
